In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet169,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/Detection/Byteplots"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:          Banker	Number of images: 1986
Label: 1	Family:          WinExe	Number of images: 1945
Processing images ...


/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (103265280 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/usr/local/lib/python3.5/dist-packages/PIL/Image.py:2438: DecompressionBombWarning: Image size (110601216 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


Images processed: 3931


In [4]:
X.shape

(3931, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet169(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(3931, 1, 1, 1664)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(3931, 1664)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 5  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 5 folds for each of 13 candidates, totalling 65 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 ...........................................................
[CV] C=1.0 .....

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    4.3s


[CV] ........................... C=0.01, score=0.941476, total=   4.8s
[CV] ........................... C=0.01, score=0.964377, total=   4.9s
[CV] C=1000.0 ........................................................
[CV] C=1000.0 ........................................................
[CV] ........................... C=0.01, score=0.947837, total=   5.6s
[CV] C=1000.0 ........................................................
[CV] ........................... C=10.0, score=0.931298, total=   9.9s
[CV] C=1000.0 ........................................................
[CV] ............................ C=0.1, score=0.936387, total=  11.1s
[CV] ............................ C=1.0, score=0.931298, total=  10.9s
[CV] C=10000.0 .......................................................
[CV] C=10000.0 .......................................................
[CV] ............................ C=0.1, score=0.946633, total=  11.6s
[CV] C=10000.0 .......................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   12.8s


[CV] .......................... C=100.0, score=0.931298, total=  11.2s
[CV] C=100000.0 ......................................................
[CV] ............................ C=0.1, score=0.950382, total=  12.6s
[CV] ............................ C=1.0, score=0.937738, total=  12.4s
[CV] C=100000.0 ......................................................
[CV] .......................... C=100.0, score=0.940204, total=  11.4s
[CV] C=1000000.0 .....................................................
[CV] .......................... C=100.0, score=0.933926, total=  11.5s
[CV] ............................ C=0.1, score=0.940204, total=  12.4s
[CV] C=1000000.0 .....................................................
[CV] C=1000000.0 .....................................................
[CV] ........................... C=10.0, score=0.924936, total=  12.0s
[CV] ............................ C=1.0, score=0.938931, total=  12.5s
[CV] ........................... C=10.0, score=0.937659, total=  12.2s
[CV] C

...

[CV] ................... C=1000000000.0, score=0.931298, total=   6.9s
[CV] .................. C=10000000000.0, score=0.931298, total=   6.1s
[CV] .................. C=10000000000.0, score=0.940204, total=   6.1s
[CV] .................. C=10000000000.0, score=0.924936, total=   5.9s
[CV] .................... C=100000000.0, score=0.924936, total=   8.7s
[CV] .................... C=100000000.0, score=0.938931, total=   8.7s
[CV] ................... C=1000000000.0, score=0.924936, total=   8.1s
[CV] ................... C=1000000000.0, score=0.935197, total=   8.8s


[Parallel(n_jobs=-1)]: Done  60 out of  65 | elapsed:   32.0s remaining:    2.7s


[CV] ................... C=1000000000.0, score=0.938931, total=   8.0s
[CV] ................... C=1000000000.0, score=0.940204, total=   8.5s
[CV] .................. C=10000000000.0, score=0.938931, total=   7.5s
[CV] .................. C=10000000000.0, score=0.935197, total=   8.6s


[Parallel(n_jobs=-1)]: Done  65 out of  65 | elapsed:   32.8s finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([ 4.57117591, 11.86078773, 11.68510485, 11.29738855, 11.05193372,
        10.78888059, 11.04623499, 11.30727601, 11.09674654, 11.35875626,
         9.96929636,  8.01435938,  6.82298274]),
 'mean_score_time': array([0.05468793, 0.06295457, 0.06007857, 0.06333752, 0.0561367 ,
        0.07028642, 0.0520339 , 0.04218688, 0.06160097, 0.0523221 ,
        0.0449513 , 0.04432764, 0.00911598]),
 'mean_test_score': array([0.95370135, 0.94352582, 0.93640295, 0.93436785, 0.93385907,
        0.93462223, 0.93411346, 0.93411346, 0.93411346, 0.93411346,
        0.93411346, 0.93411346, 0.93411346]),
 'mean_train_score': array([0.98549974, 0.99777406, 0.99993641, 0.99987279, 0.99993641,
        0.99993641, 0.99993641, 0.99993641, 0.99993641, 0.99993641,
        0.99993641, 0.99993641, 0.99993641]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
                    1000000

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.95370, std: 0.00808, params: {'C': 0.01},
 mean: 0.94353, std: 0.00488, params: {'C': 0.1},
 mean: 0.93640, std: 0.00448, params: {'C': 1.0},
 mean: 0.93437, std: 0.00556, params: {'C': 10.0},
 mean: 0.93386, std: 0.00552, params: {'C': 100.0},
 mean: 0.93462, std: 0.00408, params: {'C': 1000.0},
 mean: 0.93411, std: 0.00390, params: {'C': 10000.0},
 mean: 0.93411, std: 0.00390, params: {'C': 100000.0},
 mean: 0.93411, std: 0.00390, params: {'C': 1000000.0},
 mean: 0.93411, std: 0.00554, params: {'C': 10000000.0},
 mean: 0.93411, std: 0.00554, params: {'C': 100000000.0},
 mean: 0.93411, std: 0.00554, params: {'C': 1000000000.0},
 mean: 0.93411, std: 0.00554, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9537
